# Loop 15 Analysis: Understanding the Gap

## Key Findings
- Current score: 70.630478
- Target: 68.919154
- Gap: 1.711 (2.42%)
- Theoretical lower bound at 70% efficiency: 70.178571
- Target requires ~71.3% packing efficiency

In [1]:
import pandas as pd
import numpy as np
import os
import glob

# Check all available CSV sources
print("Available CSV sources:")
for path in glob.glob('/home/code/exploration/datasets/**/*.csv', recursive=True):
    print(f"  {path}")
for path in glob.glob('/home/code/exploration/datasets/*.csv'):
    print(f"  {path}")

Available CSV sources:
  /home/code/exploration/datasets/submission (77).csv
  /home/code/exploration/datasets/current_best.csv
  /home/code/exploration/datasets/submission.csv
  /home/code/exploration/datasets/submission (80).csv
  /home/code/exploration/datasets/ensemble_best.csv
  /home/code/exploration/datasets/santa-2025.csv
  /home/code/exploration/datasets/saspav_best.csv
  /home/code/exploration/datasets/submission_best.csv
  /home/code/exploration/datasets/my_optimized_submission.csv.csv
  /home/code/exploration/datasets/72.49.csv
  /home/code/exploration/datasets/71.97.csv
  /home/code/exploration/datasets/smartmanoj.csv
  /home/code/exploration/datasets/telegram/72.49.csv
  /home/code/exploration/datasets/telegram/71.97.csv
  /home/code/exploration/datasets/saspav_csv/santa-2025.csv
  /home/code/exploration/datasets/solutions/submission_70.926137.csv
  /home/code/exploration/datasets/seowoohyeon/submission.csv
  /home/code/exploration/datasets/seowoohyeon/submission_sa.csv
 

In [2]:
# Check snapshots for any unexplored sources
import subprocess
result = subprocess.run(['ls', '-la', '/home/nonroot/snapshots/santa-2025/'], capture_output=True, text=True)
print("Snapshots available:")
print(result.stdout[:2000])

Snapshots available:
total 284
drwxr-xr-x 71 nonroot nonroot 4096 Jan 20 22:58 .
drwxr-xr-x  4 nonroot nonroot 4096 Jan 20 17:05 ..
drwxr-xr-x  5 nonroot nonroot 4096 Jan 20 17:05 20952569566
drwxr-xr-x  5 nonroot nonroot 4096 Jan 20 17:05 20970671503
drwxr-xr-x  5 nonroot nonroot 4096 Jan 20 17:05 20971964134
drwxr-xr-x  5 nonroot nonroot 4096 Jan 20 17:05 20984924920
drwxr-xr-x  5 nonroot nonroot 4096 Jan 20 17:05 20991308120
drwxr-xr-x  5 nonroot nonroot 4096 Jan 20 17:05 20992150197
drwxr-xr-x  5 nonroot nonroot 4096 Jan 20 17:05 20992536951
drwxr-xr-x  5 nonroot nonroot 4096 Jan 20 17:05 21016257921
drwxr-xr-x  5 nonroot nonroot 4096 Jan 20 17:05 21086827828
drwxr-xr-x  5 nonroot nonroot 4096 Jan 20 17:05 21090949260
drwxr-xr-x  5 nonroot nonroot 4096 Jan 20 17:05 21104669204
drwxr-xr-x  5 nonroot nonroot 4096 Jan 20 17:05 21105319338
drwxr-xr-x  4 nonroot nonroot 4096 Jan 20 17:05 21106484795
drwxr-xr-x  5 nonroot nonroot 4096 Jan 20 17:05 21108486172
drwxr-xr-x  5 nonroot nonroo

In [3]:
# Load and analyze a few snapshots to find any with better scores
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon

getcontext().prec = 25
scale_factor = Decimal("1e15")

class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)
        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w = Decimal("0.4")
        top_w = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h
        initial_polygon = Polygon([
            (Decimal("0.0") * scale_factor, tip_y * scale_factor),
            (top_w / Decimal("2") * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal("4") * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal("2") * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal("4") * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal("2") * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal("2") * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal("2") * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal("2")) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal("2")) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal("2")) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal("4")) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal("2")) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal("4")) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal("2")) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor),
        )

def calculate_score(trees):
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    return max(max_x - min_x, max_y - min_y) ** 2 / len(trees)

def has_collision(trees):
    if len(trees) <= 1:
        return False
    for i in range(len(trees)):
        for j in range(i+1, len(trees)):
            if trees[i].polygon.intersects(trees[j].polygon) and not trees[i].polygon.touches(trees[j].polygon):
                return True
    return False

def load_trees(n, df):
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"]).lstrip('sx')
        y = str(row["y"]).lstrip('sy')
        deg = str(row["deg"]).lstrip('sd')
        trees.append(ChristmasTree(x, y, deg))
    return trees

print("Functions defined")

Functions defined


In [4]:
# Analyze current best solution per-N
df = pd.read_csv('/home/code/exploration/datasets/saspav_best.csv')

scores = {}
for n in range(1, 201):
    trees = load_trees(n, df)
    scores[n] = calculate_score(trees)

print(f"Total score: {sum(scores.values()):.6f}")
print(f"Target: 68.919154")
print(f"Gap: {sum(scores.values()) - 68.919154:.6f}")

# Find N values with most potential for improvement
print("\nN values with highest scores (most room for improvement):")
sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
for n, score in sorted_scores[:20]:
    print(f"  N={n:3d}: score={score:.6f}")

Total score: 70.630478
Target: 68.919154
Gap: 1.711324

N values with highest scores (most room for improvement):
  N=  1: score=0.661250
  N=  2: score=0.450779
  N=  3: score=0.434745
  N=  5: score=0.416850
  N=  4: score=0.416545
  N=  7: score=0.399897
  N=  6: score=0.399610
  N=  9: score=0.387415
  N=  8: score=0.385407
  N= 15: score=0.376978
  N= 10: score=0.376630
  N= 21: score=0.376451
  N= 20: score=0.376057
  N= 22: score=0.375258
  N= 11: score=0.374924
  N= 16: score=0.374128
  N= 26: score=0.373997
  N= 12: score=0.372724
  N= 13: score=0.372294
  N= 25: score=0.372144


In [5]:
# Check if there are any unexplored snapshots with better per-N scores
import os

snapshot_dir = '/home/nonroot/snapshots/santa-2025/'
snapshot_ids = os.listdir(snapshot_dir)

print(f"Found {len(snapshot_ids)} snapshots")

# Sample a few to check their scores
better_n_values = {}
for snap_id in snapshot_ids[:10]:  # Check first 10
    csv_path = os.path.join(snapshot_dir, snap_id, 'submission.csv')
    if os.path.exists(csv_path):
        try:
            snap_df = pd.read_csv(csv_path)
            for n in [50, 100, 150, 200]:  # Check a few N values
                trees = load_trees(n, snap_df)
                if len(trees) == n:
                    snap_score = calculate_score(trees)
                    if snap_score < scores[n] - 0.0001:  # Better by at least 0.0001
                        if not has_collision(trees):
                            if n not in better_n_values or snap_score < better_n_values[n][1]:
                                better_n_values[n] = (snap_id, snap_score)
                                print(f"Found better N={n}: {snap_score:.6f} vs {scores[n]:.6f} in {snap_id}")
        except Exception as e:
            pass

print(f"\nFound {len(better_n_values)} N values with better scores in snapshots")

Found 69 snapshots

Found 0 N values with better scores in snapshots


In [ ]:
# Check ALL snapshots more thoroughly
import os

snapshot_dir = '/home/nonroot/snapshots/santa-2025/'
snapshot_ids = os.listdir(snapshot_dir)

print(f"Checking all {len(snapshot_ids)} snapshots...")

# Track best scores per N across all snapshots
best_per_n = {n: (None, scores[n]) for n in range(1, 201)}  # (source, score)

for snap_id in snapshot_ids:
    csv_path = os.path.join(snapshot_dir, snap_id, 'submission.csv')
    if os.path.exists(csv_path):
        try:
            snap_df = pd.read_csv(csv_path)
            for n in range(1, 201):
                trees = load_trees(n, snap_df)
                if len(trees) == n:
                    snap_score = calculate_score(trees)
                    if snap_score < best_per_n[n][1] - 1e-9:  # Better
                        if not has_collision(trees):
                            best_per_n[n] = (snap_id, snap_score)
        except Exception as e:
            pass

# Count improvements
improvements = [(n, src, score, scores[n]) for n, (src, score) in best_per_n.items() if src is not None]
print(f"\nFound {len(improvements)} N values with better scores in snapshots:")
for n, src, new_score, old_score in sorted(improvements, key=lambda x: x[3] - x[2], reverse=True)[:20]:
    print(f"  N={n:3d}: {old_score:.6f} -> {new_score:.6f} (improvement: {old_score - new_score:.6f}) from {src}")